## Multiscale analysis of spatial correlation between 2 signals

In [ ]:
from tapenade.analysis.spatial_correlation import (
    SpatialCorrelationPlotter
)
from pathlib import Path
import tifffile
import numpy as np
from tapenade.preprocessing._preprocessing import change_array_pixelsize,normalize_intensity

### 1. Data loading

Load your the two signals that you wish to study the correlation between. If raw image data is loaded, the correlation will be studied at the smallest scale (voxel). To study the correlation at larger scales, we recommend applying our masked gaussian filter method, as presented in the `masked_gaussian_smoothing` notebook.

In [ ]:
folder = folder = Path(globals()['_dh'][0]).parents[2] / 'data'
path_to_signal = Path(folder) / '2k_Hoechst_FoxA2_Oct4_Bra_78h/big/data/1.tif'
path_to_mask =  Path(folder) / '2k_Hoechst_FoxA2_Oct4_Bra_78h/big/masks/1_mask.tif'
path_to_labels = Path(folder) / '2k_Hoechst_FoxA2_Oct4_Bra_78h/big/segmentation/1_seg.tif'
mask = tifffile.imread(path_to_mask)
labels = tifffile.imread(path_to_labels)
signal = tifffile.imread(path_to_signal)
hoechst = signal[:,0,:,:]
foxa2 = signal[:,1,:,:]
oct4 = signal[:,2,:,:]
bra = signal[:,3,:,:]

In [ ]:
oct4 = change_array_pixelsize(oct4, input_pixelsize=(1,0.6,0.6))
foxa2 = change_array_pixelsize(foxa2, input_pixelsize=(1,0.6,0.6))
hoechst = change_array_pixelsize(hoechst, input_pixelsize=(1,0.6,0.6))
mask = (change_array_pixelsize(mask, input_pixelsize=(1,0.6,0.6),order=0)).astype(int)
labels = (change_array_pixelsize(labels, input_pixelsize=(1,0.6,0.6),order=0)).astype(int)

foxa2_norm = normalize_intensity(foxa2,ref_image=hoechst,image_wavelength=488,sigma=11)
oct4_norm = normalize_intensity(oct4,ref_image=hoechst,image_wavelength=555,sigma=11)

In [ ]:
import napari
viewer = napari.Viewer()
viewer.add_image(foxa2*mask,name='foxa2',colormap='gray_r')
viewer.add_image(oct4*mask,name='Oct4',colormap='gray_r')
viewer.add_image(foxa2_norm*mask,name='foxa2_norm',colormap='gray_r')
viewer.add_image(oct4_norm*mask,name='oct4_norm',colormap='gray_r')

for l in viewer.layers:
    l.data = np.transpose(l.data, (1, 0, 2))

viewer.grid.enabled = True
viewer.grid.shape = (2, 2)
viewer.grid.stride = -1

napari.run()

### 2. Initializing the plotter

Initializing the plotter once allows you to test several parameters of the visualization without reprocessing the data.

In [ ]:
spatial_correlation_plotter_oct4_foxa2_norm = SpatialCorrelationPlotter(
    quantity_X=oct4_norm,
    quantity_Y=foxa2_norm,
    mask=mask,
    labels=labels,

)
spatial_correlation_plotter_oct4_foxa2 = SpatialCorrelationPlotter(
    quantity_X=oct4,
    quantity_Y=foxa2,
    mask=mask,
    labels=labels,

)

Plot the correlation:

In [ ]:
lim_foxa2=200
lim_oct4=80
bins=70
fig, ax = spatial_correlation_plotter_oct4_foxa2.get_heatmap_figure(
    show_linear_fit=False,
    show_individual_cells=True,
    display_quadrants=True,
    bins=[bins,bins],
    extent_X = (0,lim_oct4),
    extent_Y = (0,lim_foxa2),
    label_X='Oct4',
    label_Y='Foxa2',
)
# plt.savefig(Path(folder)/ 'S8g_unnormalized.png',dpi=300)
fig, ax = spatial_correlation_plotter_oct4_foxa2_norm.get_heatmap_figure(
    show_linear_fit=False,
    
    show_individual_cells=True,
    display_quadrants=True,
    bins=[bins,bins],
    extent_X = (0,lim_oct4),
    extent_Y = (0,lim_foxa2),
    label_X='Oct4',
    label_Y='Foxa2',
)

fig.show()
# plt.savefig(Path(folder)/ 'S8g_normalized.png',dpi=300)